## Sparsity feature selector

- We select the top-k features for a given dataset X (N_samples x N_feats). X is expected to be binary. X[i,j] = 1 denotes that the feature j for sample i is not missing while X[i,j] = 0 denotes a missing feature.
- The resulting subset of features will have all th

In [226]:
import cvxpy as cp
import numpy as np

# N_samples X N_features matrix with binary values denoting whether the feature exists
X = np.random.randint(2, size=(1000, 60))
#X = np.vstack((np.ones((1000,60)), np.hstack((np.zeros((1000,30)), np.ones((1000,30)))), np.random.randint(2, size=(1000,60))))

print(f'Sparsity of features:\n{100 * (1 - (X.sum(axis=0) / len(X)))}')
N_samples = X.shape[0]
N_features = X.shape[1]

k = cp.Constant(2)

# # The costs for each feature
w = cp.Constant(X)
# Variables containing the number of the coins to be returned for each denominator
# The size of this must be equal to the denominations w
x = cp.Variable((1, N_features), boolean=True)

# We want to minimize the total number of coins returned
objective = cp.Minimize(cp.sum(k - w @ x.T))



# The constraints
constraints = [
    cp.sum(x) == k,
    x>=0 # semi-positive coins
]
# Form and solve problem.
prob = cp.Problem(objective, constraints)
# Need the GLPK_MI solver because the ECOS_BB is not working correctly.
prob.solve(solver = 'GLPK_MI') # Returns the optimal value.
if prob.status == 'infeasible':
    print("Can't change %s with denominations: %s"%(CASH.__str__(), w.__str__()))
else:
    #return
    res = x.value.flatten()
    keeping = np.sum(X[:, res.astype(bool)].sum(axis=1)==k.value)
    print(f"Best result with keeping: {int(keeping)} / {N_samples} ({100 * (int(keeping)) / N_samples:.2f} %)")
    print(f"Best feature indices: {res}")
#  Keeping only the selected features
subset = X[:, res.astype(bool)]
# Dropping non-zero rows
subset = subset[np.all(subset, axis=1)]
print(f'Keeping: {subset.shape}')

Sparsity of features:
[50.4 51.7 50.3 53.2 50.7 51.2 51.5 52.7 50.  51.8 50.4 49.4 47.2 50.6
 49.1 51.6 49.7 50.9 48.7 49.4 48.2 48.3 50.4 51.5 52.6 52.9 50.7 50.2
 50.6 51.4 51.4 49.4 52.5 49.3 47.3 48.6 52.9 52.4 51.9 49.6 52.1 50.7
 48.9 48.6 50.6 49.3 50.2 50.2 51.9 48.8 52.4 50.9 51.1 50.2 49.7 49.
 51.8 50.4 49.2 50.5]
Best result with keeping: 272 / 1000 (27.20 %)
Best feature indices: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Keeping: (272, 2)


/home/kbogas/anaconda3/lib/python3.7/site-packages/cvxpy/interface/numpy_interface/ndarray_interface.py:47: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if result.dtype in [numpy.complex, numpy.float64]:


In [194]:
def find_best_feats(X, top_k, print_=False):
    if print_:
        print(f'Sparsity of features:\n{100 * (1 - (X.sum(axis=0) / len(X)))}')
    N_samples = X.shape[0]
    N_features = X.shape[1]

    k = cp.Constant(top_k)

    # # The costs for each feature
    w = cp.Constant(X)
    # Variables containing the number of the coins to be returned for each denominator
    # The size of this must be equal to the denominations w
    x = cp.Variable((1, N_features), boolean=True)

    # We want to minimize the total number of coins returned
    objective = cp.Minimize(cp.sum(k - w @ x.T))



    # The constraints
    constraints = [
        cp.sum(x) == k,
        x>=0 # semi-positive coins
    ]
    # Form and solve problem.
    prob = cp.Problem(objective, constraints)
    # Need the GLPK_MI solver because the ECOS_BB is not working correctly.
    prob.solve(solver = 'GLPK_MI') # Returns the optimal value.
    if prob.status == 'infeasible':
        print("Can't change %s with denominations: %s"%(CASH.__str__(), w.__str__()))
        raise Error
        return []
    else:
        #return
        res = x.value.flatten()
        keeping = np.sum(X[:, res.astype(bool)].sum(axis=1)==k.value)
        if print_:
            print(f"Best result with keeping: {int(keeping)} / {N_samples} ({100 * (int(keeping)) / N_samples:.2f} %)")
            print(f"Best feature indices: {res}")
        return res

In [219]:
X = np.random.randint(2, size=(1000, 60))
perc = []
for k in range(1, X.shape[1]):
    res = find_best_feats(X, k)
    subset = X[:, res.astype(bool)]
    keeping = np.sum(subset.sum(axis=1)==k)
    print(f"# features {k} | Result: {int(keeping)} / {N_samples} ({100 * (int(keeping)) / N_samples:.2f} %)")
    print(f'X shape: {subset[np.all(subset, axis=1)].shape}')
    perc.append(int(keeping) / N_samples)
    #break
    

/home/kbogas/anaconda3/lib/python3.7/site-packages/cvxpy/interface/numpy_interface/ndarray_interface.py:47: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if result.dtype in [numpy.complex, numpy.float64]:


# features 1 | Result: 532 / 1000 (53.20 %)
X shape: (532, 1)
# features 2 | Result: 283 / 1000 (28.30 %)
X shape: (283, 2)
# features 3 | Result: 129 / 1000 (12.90 %)
X shape: (129, 3)
# features 4 | Result: 69 / 1000 (6.90 %)
X shape: (69, 4)
# features 5 | Result: 31 / 1000 (3.10 %)
X shape: (31, 5)
# features 6 | Result: 16 / 1000 (1.60 %)
X shape: (16, 6)
# features 7 | Result: 4 / 1000 (0.40 %)
X shape: (4, 7)
# features 8 | Result: 2 / 1000 (0.20 %)
X shape: (2, 8)
# features 9 | Result: 0 / 1000 (0.00 %)
X shape: (0, 9)
# features 10 | Result: 0 / 1000 (0.00 %)
X shape: (0, 10)
# features 11 | Result: 0 / 1000 (0.00 %)
X shape: (0, 11)
# features 12 | Result: 0 / 1000 (0.00 %)
X shape: (0, 12)
# features 13 | Result: 0 / 1000 (0.00 %)
X shape: (0, 13)
# features 14 | Result: 0 / 1000 (0.00 %)
X shape: (0, 14)
# features 15 | Result: 0 / 1000 (0.00 %)
X shape: (0, 15)
# features 16 | Result: 0 / 1000 (0.00 %)
X shape: (0, 16)
# features 17 | Result: 0 / 1000 (0.00 %)
X shape: (0

array([], shape=(0, 60), dtype=int64)

255

In [80]:
N_samples - np.count_nonzero((X @ x.T).reshape(-1,) - k)

241

In [96]:
np.dot(X, x.T).shape

(1000, 1)

In [55]:
np.count_nonzero((X @ np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]).reshape(1,-1).T - k))

785

In [141]:
X[:, x.value.flatten().astype(bool)]

array([[1, 0, 1, ..., 1, 0, 1],
       [0, 1, 0, ..., 1, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 1, ..., 0, 0, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 0]])

In [ ]:
lambda_ = 0.5

def l
cp.sum(x) * lambda_ + lambda_*loss()